In [2]:
# 1. Fix imports
import sys, os
sys.path.append(os.path.abspath(".."))

import pandas as pd
from src.data_processing import task4_create_proxy_target

# 2. Load data
df = pd.read_csv("../data/raw/data.csv")

# 3. Run Task-4
df_out = task4_create_proxy_target(df)

# 4. Verify output
df_out[['CustomerId', 'is_high_risk']].head()


,CustomerId,is_high_risk
0,CustomerId_4406,0
1,CustomerId_4406,0
2,CustomerId_4683,0
3,CustomerId_988,0
4,CustomerId_988,0


In [3]:
df_out['is_high_risk'].value_counts()


is_high_risk
0    91571
1     4091
Name: count, dtype: int64

In [5]:
df_out.columns


Index(['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId',
       'CurrencyCode', 'CountryCode', 'ProviderId', 'ProductId',
       'ProductCategory', 'ChannelId', 'Amount', 'Value',
       'TransactionStartTime', 'PricingStrategy', 'FraudResult',
       'is_high_risk'],
      dtype='object')

In [6]:
import pandas as pd

snapshot_date = pd.to_datetime(df_out['TransactionStartTime']).max() + pd.Timedelta(days=1)

rfm_check = (
    df_out
    .groupby('CustomerId')
    .agg(
        Recency=('TransactionStartTime',
                 lambda x: (snapshot_date - pd.to_datetime(x).max()).days),
        Frequency=('TransactionId', 'count'),
        MonetarySum=('Amount', 'sum'),
        is_high_risk=('is_high_risk', 'max')
    )
)

rfm_check.groupby('is_high_risk')[['Recency','Frequency','MonetarySum']].mean()


,Recency,Frequency,MonetarySum
is_high_risk,,,
0,31.461909,24.47768,1.998243e+05
1,29.000000,4091.00000,-1.049000e+08


In [7]:
# Import libraries
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Ensure your DataFrame is loaded, e.g.,
# df = pd.read_csv("data/processed/transactions.csv")

# Make sure TransactionStartTime is datetime
df['TransactionStartTime'] = pd.to_datetime(df['TransactionStartTime'])

# Define snapshot date (max date + 1 day)
snapshot_date = df['TransactionStartTime'].max() + pd.Timedelta(days=1)

# Calculate RFM metrics
rfm = df.groupby('CustomerId').agg(
    Recency=('TransactionStartTime', lambda x: (snapshot_date - x.max()).days),
    Frequency=('TransactionId', 'count'),
    MonetarySum=('Amount', 'sum')
).reset_index()

# Scale RFM features
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm[['Recency','Frequency','MonetarySum']])

# Cluster customers using KMeans
kmeans = KMeans(n_clusters=3, random_state=42)
rfm['Cluster'] = kmeans.fit_predict(rfm_scaled)

# Identify the high-risk cluster (lowest Frequency & MonetarySum)
cluster_means = rfm.groupby('Cluster')[['Recency','Frequency','MonetarySum']].mean()
high_risk_cluster = cluster_means.sort_values(['Frequency','MonetarySum']).index[0]

# Assign binary target
rfm['is_high_risk'] = (rfm['Cluster'] == high_risk_cluster).astype(int)

# Merge target back to original DataFrame
df_out = df.merge(rfm[['CustomerId','is_high_risk']], on='CustomerId', how='left')

# Quick check
df_out[['CustomerId', 'is_high_risk']].head()


,CustomerId,is_high_risk
0,CustomerId_4406,0
1,CustomerId_4406,0
2,CustomerId_4683,1
3,CustomerId_988,0
4,CustomerId_988,0


In [8]:
import pandas as pd
from datetime import datetime

# Example df
# df = pd.read_csv("data/processed/transactions.csv")  # your main transaction data

snapshot_date = df['TransactionStartTime'].max() + pd.Timedelta(days=1)  # define snapshot date
df['TransactionStartTime'] = pd.to_datetime(df['TransactionStartTime'])

rfm = df.groupby('CustomerId').agg({
    'TransactionStartTime': lambda x: (snapshot_date - x.max()).days,
    'TransactionId': 'count',
    'Amount': 'sum'
}).rename(columns={
    'TransactionStartTime': 'Recency',
    'TransactionId': 'Frequency',
    'Amount': 'Monetary'
}).reset_index()

rfm.head()


,CustomerId,Recency,Frequency,Monetary
0,CustomerId_1,84,1,-10000.0
1,CustomerId_10,84,1,-10000.0
2,CustomerId_1001,90,5,20000.0
3,CustomerId_1002,26,11,4225.0
4,CustomerId_1003,12,6,20000.0


In [10]:
# Define snapshot date (e.g., one day after the last transaction)
snapshot_date = df['TransactionStartTime'].max()
snapshot_date = pd.to_datetime(snapshot_date) + pd.Timedelta(days=1)

# Convert transaction time to datetime
df['TransactionStartTime'] = pd.to_datetime(df['TransactionStartTime'])

# Aggregate metrics
rfm = df.groupby('CustomerId').agg({
    'TransactionStartTime': lambda x: (snapshot_date - x.max()).days,  # Recency
    'TransactionId': 'count',  # Frequency
    'Amount': 'sum'  # Monetary
}).reset_index()

rfm.rename(columns={
    'TransactionStartTime': 'Recency',
    'TransactionId': 'Frequency',
    'Amount': 'Monetary'
}, inplace=True)

rfm.head()


,CustomerId,Recency,Frequency,Monetary
0,CustomerId_1,84,1,-10000.0
1,CustomerId_10,84,1,-10000.0
2,CustomerId_1001,90,5,20000.0
3,CustomerId_1002,26,11,4225.0
4,CustomerId_1003,12,6,20000.0


In [11]:
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm[['Recency', 'Frequency', 'Monetary']])


In [12]:
kmeans = KMeans(n_clusters=3, random_state=42)
rfm['Cluster'] = kmeans.fit_predict(rfm_scaled)


In [13]:
# Inspect cluster centers
cluster_centers = pd.DataFrame(scaler.inverse_transform(kmeans.cluster_centers_), 
                               columns=['Recency', 'Frequency', 'Monetary'])
print(cluster_centers)

# Assume the cluster with highest Recency and lowest Frequency/Monetary is high-risk
high_risk_cluster = cluster_centers['Recency'].idxmax()
print(f"High-risk cluster: {high_risk_cluster}")

# Assign binary target
rfm['is_high_risk'] = (rfm['Cluster'] == high_risk_cluster).astype(int)
rfm.head()


     Recency    Frequency      Monetary
0  61.485497     7.647790  8.166357e+04
1  29.000000  4091.000000 -1.049000e+08
2  12.502399    35.105539  2.744412e+05
High-risk cluster: 0


,CustomerId,Recency,Frequency,Monetary,Cluster,is_high_risk
0,CustomerId_1,84,1,-10000.0,0,1
1,CustomerId_10,84,1,-10000.0,0,1
2,CustomerId_1001,90,5,20000.0,0,1
3,CustomerId_1002,26,11,4225.0,2,0
4,CustomerId_1003,12,6,20000.0,2,0


In [14]:
df = df.merge(rfm[['CustomerId', 'is_high_risk']], on='CustomerId', how='left')
df.head()


,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,is_high_risk
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15 02:18:49+00:00,2,0,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15 02:19:08+00:00,2,0,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15 02:44:21+00:00,2,0,1
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15 03:32:55+00:00,2,0,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15 03:34:21+00:00,2,0,0


In [1]:
# =========================
# Task-4: Proxy Target Verification
# =========================

import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# -----------------------------
# Step 1: Load data
# -----------------------------
# Use relative path from notebooks/ folder
df = pd.read_csv("../data/raw/data.csv")
print("Data shape:", df.shape)
print(df.head())

# -----------------------------
# Step 2: Define Task-4 function
# -----------------------------
def task4_create_proxy_target(df: pd.DataFrame, snapshot_date: str = None) -> pd.DataFrame:
    df = df.copy()
    df['TransactionStartTime'] = pd.to_datetime(df['TransactionStartTime'])

    if snapshot_date is None:
        snapshot_date = df['TransactionStartTime'].max() + pd.Timedelta(days=1)
    else:
        snapshot_date = pd.to_datetime(snapshot_date)

    # RFM calculation
    rfm = (
        df.groupby('CustomerId')
        .agg(
            Recency=('TransactionStartTime', lambda x: (snapshot_date - x.max()).days),
            Frequency=('TransactionId', 'count'),
            Monetary=('Amount', 'sum')
        )
        .reset_index()
    )

    # Scale
    scaler = StandardScaler()
    rfm_scaled = scaler.fit_transform(rfm[['Recency', 'Frequency', 'Monetary']])

    # KMeans clustering
    kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
    rfm['cluster'] = kmeans.fit_predict(rfm_scaled)

    # Identify high-risk cluster (least engaged)
    cluster_stats = rfm.groupby('cluster')[['Frequency', 'Monetary']].mean()
    high_risk_cluster = cluster_stats.sum(axis=1).idxmin()

    # Create target
    rfm['is_high_risk'] = (rfm['cluster'] == high_risk_cluster).astype(int)

    # Merge back
    df = df.merge(
        rfm[['CustomerId', 'is_high_risk']],
        on='CustomerId',
        how='left'
    )
    return df

# -----------------------------
# Step 3: Apply function
# -----------------------------
df_with_target = task4_create_proxy_target(df)
print("\nHigh-risk column added:")
print(df_with_target[['CustomerId', 'is_high_risk']].head())

# -----------------------------
# Step 4: Verify distribution
# -----------------------------
print("\nHigh-risk distribution:")
print(df_with_target['is_high_risk'].value_counts())

# -----------------------------
# Step 5: Optional: Inspect clusters
# -----------------------------
rfm_check = df_with_target.groupby('CustomerId').agg({
    'TransactionId': 'count',
    'Amount': 'sum',
    'is_high_risk': 'max'
}).reset_index()

print("\nSample of RFM + high-risk label:")
print(rfm_check.sort_values(by='is_high_risk', ascending=False).head(10))


Data shape: (95662, 16)
         TransactionId         BatchId       AccountId       SubscriptionId  \
0  TransactionId_76871   BatchId_36123  AccountId_3957   SubscriptionId_887   
1  TransactionId_73770   BatchId_15642  AccountId_4841  SubscriptionId_3829   
2  TransactionId_26203   BatchId_53941  AccountId_4229   SubscriptionId_222   
3    TransactionId_380  BatchId_102363   AccountId_648  SubscriptionId_2185   
4  TransactionId_28195   BatchId_38780  AccountId_4841  SubscriptionId_3829   

        CustomerId CurrencyCode  CountryCode    ProviderId     ProductId  \
0  CustomerId_4406          UGX          256  ProviderId_6  ProductId_10   
1  CustomerId_4406          UGX          256  ProviderId_4   ProductId_6   
2  CustomerId_4683          UGX          256  ProviderId_6   ProductId_1   
3   CustomerId_988          UGX          256  ProviderId_1  ProductId_21   
4   CustomerId_988          UGX          256  ProviderId_4   ProductId_6   

      ProductCategory    ChannelId   Amount 

In [2]:
# Save processed data
df_with_target.to_csv("../data/processed/data_with_target.csv", index=False)


In [10]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Load data
df = pd.read_csv("../data/raw/data.csv")

df["TransactionStartTime"] = pd.to_datetime(df["TransactionStartTime"])

# Snapshot date
snapshot_date = df["TransactionStartTime"].max() + pd.Timedelta(days=1)

# Create RFM
rfm = df.groupby("CustomerId").agg({
    "TransactionStartTime": lambda x: (snapshot_date - x.max()).days,
    "TransactionId": "count",
    "Amount": "sum"
}).reset_index()

rfm.columns = ["CustomerId", "Recency", "Frequency", "Monetary"]
rfm["Monetary"] = rfm["Monetary"].abs()

# Scale
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm[["Recency", "Frequency", "Monetary"]])

# KMeans
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
rfm["cluster"] = kmeans.fit_predict(rfm_scaled)

# ✅ NOW this works
cluster_counts = rfm["cluster"].value_counts().sort_index()
print("Number of customers in each cluster:")
print(cluster_counts)


Number of customers in each cluster:
cluster
0    1426
1    2312
2       4
Name: count, dtype: int64


In [11]:
print("Transactions:", df.shape[0])
print("Unique customers:", df["CustomerId"].nunique())
print("RFM rows:", rfm.shape[0])


Transactions: 95662
Unique customers: 3742
RFM rows: 3742
